In [ ]:
%pip install datasets huggingface-cli

This dataset is big and not properly formatted for HF. I found it more reliable to first download the dataset with the HF CLI
   ```bash
   # Create datasets directory (it's gitignored)
   mkdir datasets

   # Download the dataset
   huggingface-cli download --repo-type dataset filapro/cad-recode --local-dir datasets/cad-recode
   ```

In [1]:
# Import necessary libraries
from datasets import load_dataset

# Load from local datasets directory
dataset = load_dataset(
    "text",  # We're loading text files (Python files are text)
    data_files="datasets/cad-recode/train/batch_00/*.py",  # Full path to Python files
    split="train",
    streaming=True
)

# Look at the examples
print("\nSample examples:")
for example in dataset.take(3):  # Show first 3 examples
    print(example)

/Users/raymondweitekamp/Documents/GitHub/cad-recode-repro/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Sample examples:
{'text': 'import cadquery as cq'}
{'text': "w0=cq.Workplane('XY',origin=(0,0,-11))"}
{'text': 'r=w0.sketch().segment((12,-10),(16,-7)).arc((48,-11),(26,14)).arc((-6,7),(18,-16)).close().assemble().finalize().extrude(-13).union(w0.sketch().segment((-50,-14),(-49,-17)).segment((-43,-14)).segment((-44,-12)).close().assemble().reset().face(w0.sketch().segment((-28,-5),(-26,-8)).segment((-12,-3)).segment((-13,0)).close().assemble()).finalize().extrude(36))'}


note: cadquery and mac silicon still don't get along unfortunately...

In [ ]:
%pip install git+https://github.com/CadQuery/cadquery.git@e99a15df3cf6a88b69101c405326305b5db8ed94
%pip install trimesh 

In [ ]:
import trimesh
import numpy as np

def mesh_to_point_cloud(mesh, n_points=256):
    """Convert a mesh to a point cloud with normals"""
    vertices, faces = trimesh.sample.sample_surface(mesh, n_points)
    point_cloud = np.concatenate((
        np.asarray(vertices),
        mesh.face_normals[faces]
    ), axis=1)
    # Sort points for consistency
    ids = np.lexsort((point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2]))
    point_cloud = point_cloud[ids]
    return point_cloud

def py_string_to_mesh(py_string):
    """Convert CadQuery Python code to a mesh"""
    # Execute the Python string
    exec(py_string, globals())
    compound = globals()['r'].val()
    
    # Convert to mesh
    vertices, faces = compound.tessellate(0.001, 0.1)
    mesh = trimesh.Trimesh([(v.x, v.y, v.z) for v in vertices], faces)
    
    # Center and normalize the mesh
    mesh.apply_translation(-(mesh.bounds[0] + mesh.bounds[1]) / 2.0)
    mesh.apply_scale(2.0 / max(mesh.extents))
    
    return mesh

# Load from local datasets directory
dataset = load_dataset(
    "text",  # We're loading text files
    data_files="datasets/cad-recode/train/batch_00/*.py",  # Full path to Python files
    split="train",
    streaming=True
)

# Look at the examples and convert to point clouds
print("\nSample examples:")
for example in dataset.take(3):
    print("\nOriginal Python code:")
    print(example['text'])
    
    try:
        # Convert Python code to mesh to point cloud
        mesh = py_string_to_mesh(example['text'])
        point_cloud = mesh_to_point_cloud(mesh)
        print("\nPoint cloud shape:", point_cloud.shape)
    except Exception as e:
        print("\nError converting to point cloud:", str(e))

if you see something like this, you're on apple silicon and not going to have fun. 

```
Error converting to point cloud: dlopen(/path/to/venv/lib/python3.11/site-packages/OCP/OCP.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libvtkWrappingPythonCore3.11-9.2.dylib
  Referenced from: <CCC3E9CA-0ECE-3AA4-8D7F-735550B5EF82> /path/to/venv/lib/python3.11/site-packages/OCP/OCP.cpython-311-darwin.so
  Reason: tried: '/path/to/venv/lib/python3.11/site-packages/OCP/../vtkmodules/.dylibs/libvtkWrappingPythonCore3.11-9.2.dylib' (no such file), '/path/to/venv/lib/python3.11/site-packages/OCP/../vtkmodules/.dylibs/libvtkWrappingPythonCore3.11-9.2.dylib' (no such file), '/path/to/python/3.11.7/lib/libvtkWrappingPythonCore3.11-9.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/path/to/python/3.11.7/lib/libvtkWrappingPythonCore3.11-9.2.dylib' (no such file), '/opt/homebrew/lib/libvtkWrappingPythonCore3.11-9.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/lib/libvtkWrappingPythonCore3.11-9.2.dylib' (no such file), '/usr/local/lib/libvtkWrappingPythonCore3.11-9.2.dylib' (no such file), '/usr/lib/libvtkWrappingPythonCore3.11-9.2.dylib' (no such file, not in dyld cache
```